In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from mvn.pipeline.setup import setup_dataloaders, setup_experiment, build_env
from mvn.pipeline.training import do_train

In [3]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [6]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model (shared):
                      backbone has      19600 params (~ 35.8) %
                    R_backbone has      17590 params (~ 32.1) %
                    t_backbone has      17539 params (~ 32.0) %
total params:                54729
cam2cam estimation => adding 54729 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@17.05.2021-14:20:58
#[do_train]: epoch    0 has started!
#[cam2cam]: training batch iter 0 losses: GEO ~ 7.201, TRANS ~ 196.346, POSE ~ 78255.516, ROTO ~ 10.402, 3D ~ 617.447, SELF R ~ 69.857, SELF t ~ 22.738, TOTAL ~ 732.780
#[cam2cam]: training batch iter 0 MPJPE: ~ 440.583 mm
#[cam2cam]: training batch iter 1 losses: GEO ~ 7.118, TRANS ~ 196.953, POSE ~ 75735.023, ROTO ~ 10.411, 3D ~ 602.770, SELF R ~ 65.214, SELF t ~ 20.521, TOTAL ~ 709.026
#[cam2cam]: training batch iter 1 MPJPE: ~ 449.501 mm
#[cam2cam]: training batch iter 2 losses: GEO ~ 7.041, TRANS ~ 197.795, POS